## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


User-User рекомендации и Item-Item рекомендации отличаются тем, что в первом случае мы находим похожих юзеров, а во втором 
случае находим похожие товары. Похожи они тем, что и там и там используется история взаимодействия, а также то, что отталкиваемся
от мер схожести. Поэтому, если есть функция item_item_rec, то её можно использовать для user_user_rec. 
Принципиальные отличия item-item рекомендаций от ALS: 
    - в item-item у нас меры схожести(по факту, эвристики), в ALS похожесть это направление антиградиента при поиске 
    частной производной по юзерам/товарам)
    - в item-item у нас lazy fit, а в ALS есть лосс, который итерационно можно оптимизировать. По сути, этот лосс есть ридж-регрессия на МНК, поэтому ALS позволяет использовать l2 регуляризацию(ограничивать квадраты расстояний схожести юзеров/товаров)
    - в ALS можно получить эмбэддинги благодаря использованию свойства разложения матрицы, исходящей из линейной зависимости строк(столбцов)
    - в ALS можно не пересчитывать всё, зафиксировав эмбединги. Т.е. можно добавлять по несколько интеракций и делать на них
    предикты в реальном времени.

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


- время взаимодействия с сервисом при поиске нужного товара
- количество лайков либо какой-то рейтинг, например, средневзешенная величина лайков
- положил или нет юзер товар в корзину

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


  - В ALS нельзя добавить дополнительные признаки по юзерам/товарам
  - Разложение осуществляется на основе свойства линейной зависимости строк(столбцов)
  - Может находить только линейные взаимосвязи между взаимодействиями

4. Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

## Практическая часть


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [3]:
# import data
# utils functions like in webinar

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [42]:
data = pd.read_csv('../data/retail_train.csv')
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [43]:
result_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']

result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']

In [44]:
# Построим топ-5000
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [45]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [47]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [48]:
def precision_at_k(boughted, recommended, k=5):
    recommended = recommended[:k]

    precision = np.sum(np.isin(boughted, recommended)) / len(recommended)
    
    return precision

In [52]:
prec_train = []
prec_test = []

TF-IDF взвешивание.

In [53]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [55]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result_train['als'] = result_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_train.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
prec_train.append(prec)
prec


Wall time: 2min 32s


0.025650260104041735

In [56]:
result_test['als'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_test.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
prec_test.append(prec)
prec

0.028566764609859706

BM 25 взвешивание.

In [57]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [59]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result_train['als_bm25'] = result_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_train.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
prec_train.append(prec)
prec


Wall time: 2min 33s


0.03165266106442597

In [60]:
result_test['als_bm25'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_test.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()
prec_test.append(prec)
prec

0.03092556317335957

Попробуем BM 25 по sales value.

In [71]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', # Можно пробоват ьдругие варианты
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
user_item_matrix = bm25_weight(user_item_matrix.T).T

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [72]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result_train['als_bm25_sv'] = result_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_train.apply(lambda row: precision_at_k(row['als_bm25_sv'], row['actual']), axis=1).mean()
prec_train.append(prec)
prec


Wall time: 2min 13s


0.02918500733626799

In [73]:
result_test['als_bm25_sv'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_test.apply(lambda row: precision_at_k(row['als_bm25_sv'], row['actual']), axis=1).mean()
prec_test.append(prec)
prec

0.025457068233757818

Попробуем без доп.взвешивания по sales_value.

In [74]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', # Можно пробоват ьдругие варианты
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [75]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result_train['als_sv'] = result_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_train.apply(lambda row: precision_at_k(row['als_sv'], row['actual']), axis=1).mean()
prec_train.append(prec)
prec


Wall time: 2min 14s


0.02416299853274653

In [76]:
result_test['als_sv'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
prec = result_test.apply(lambda row: precision_at_k(row['als_sv'], row['actual']), axis=1).mean()
prec_test.append(prec)
prec

0.024673522690173078

In [77]:
approach = ['tfidf als_q', 
            'bm25 als_q',
            'bm_25_sv',
            'sv'
           ]

scores = pd.DataFrame(list(zip(approach, prec_train, prec_test)), columns=['approach','train', 'test'])
scores.set_index('approach')

,train,test
approach,,
tfidf als_q,0.025650,0.028567
bm25 als_q,0.031653,0.030926
bm_25_sv,0.038409,0.030926
sv,0.030132,0.035986


Что я думаю насчёт всего этого. Среди всех С для разработки нам необходимо различать их два вида: эмбеддинги и условно, целевой
признак(основной, "киллер фича" или что то около неё). По сути, мы изначально более-менее достоверно не понимаем, какой признак необходимо использовать в качестве
основного, чтобы рекомендовать пользователю то, что ему подходит. Чтобы выявить этот основной признак целесообразно провести
предварительный анализ поведения пользователей, их предпочтений по товарам, проанализировать товары и сформировать набор первичных 
приоритизированных гипотез о том, какие признаки могут быть основными. Вполне возможно, что этого признака нет в таблице и нам его нужно
сделать. Остальные признаки можно использовать для создания эмбедингов, чтобы улучшить результаты по основному признаку. 

Вопросы(если мои суждения выше имеют смысл):
    - может ли сработать предварительный корреляционно-регресионный анализ для того, чтобы мы поняли взаимосвязи между признаками. апример, мы установили что некоторый C должен быть основным, посмотрели корреляции с другими признаками, нашли пару 
    хорошо коррелирующих признаков. Означает ли это то, что эмбеддинг коррелирующих с С признаков будет сильным? Либо потенциально может быть сильным?

Также если взять count по sales value и sum(количество транзакций), то результаты будут разные. Мне кажется это подтверждает
мои доводы выше о том, что необходимо анализировать и работать с постановкой задачи, прежде чем выбирать главный С.

P.S. Данные нужно масштабировать.

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [79]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [80]:
precisions_train = []
precisions_test = []

In [82]:
%time

for factor in [20, 40, 60]:
    for lambda_ in [1e-3, 1e-4, 1e-6]:
        for iteration in [20, 50, 100]:
            model = AlternatingLeastSquares(factors=factor, 
                                regularization=lambda_,
                                iterations=iteration, 
                                calculate_training_loss=True, 
                                num_threads=4) # K - кол-во билжайших соседей

            model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=True)

            result_train['als'] = result_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            prec_train = result_train.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
            precisions_train.append(prec_train)
            
            result_test['als'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            prec_test = result_test.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
            precisions_test.append(prec_test)

Wall time: 0 ns


In [94]:
max(precisions_train)

0.03296651994131008

In [92]:
precisions_train.index(max(precisions_train))

25

In [93]:
precisions_test[precisions_train.index(max(precisions_train))]

0.030460333006856133

Результаты получились хуже. Нужно выставлять другую область поиска, например большее количество factors. Наверное, это логично,
поскольку у модели будет большее количество линейно зависимых строк, а значит, больше информации. Также нужно сделать регуляризацию пожёстче и снизить количество итераций.

Вопрос: такая область поиска, в целом, соответсвует только этой задаче или справедлива для любого(практически любого) ALS?
Означают ли полученные результаты то, что нужнно делать ALS консервативным и при этом, давать больше информации? Интересно было
бы узнать, почему так.

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!